In [51]:
%reset

import pandas as pd
import numpy as np
import random
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  silhouette_score
from statistics import mean 
from matplotlib import pyplot as plt
# random.seed(5)



def getData(dataset_name):
    attribute_file_name = 'Data/'+dataset_name+".attribute"
    dataset_file_name = 'Data/'+dataset_name+".data"
    att = pd.read_csv(attribute_file_name,
                      delim_whitespace=True,
                     header = None)
    attributes = {rows[0]:rows[1] for _,rows in att.iterrows()}
    dataset = pd.read_csv(dataset_file_name,
                      names=list(attributes.keys()))
    if 'class' in attributes: 
        tuple_labels = []
        label_count = []
        classes = {}
        tuple_labels = list(dataset["class"])
        classes_unique, label_count = np.unique(tuple_labels, return_counts=True)
        idx = 0
        for class_ in classes_unique:
            classes[class_] = idx
            idx += 1
#         print(classes)
        del attributes['class']; del dataset['class']
    return  dataset,classes,tuple_labels,label_count



def dist(x1,x2, minkowski = 2):
    val = 0.0
#     print("In error: ",attributes)
    for att in dataset.columns:
#         if attributes[att]=='value':
        val += (x1[att] - x2[att])*(x1[att] - x2[att])
    val = math.sqrt(val)
    return val

def calculateError():
    error = 0
    sqrt_error = 0
    quality = 0
    for i in range(len(dataset)):
        distance = dist(dataset.iloc[i], pd.DataFrame(centroids[cluster_index[i]],index=[0]))
        sqrt_error += distance
        error += distance
        quality += (1-distance)
#     print("Error:",error,"Squared Error:", sqrt_error,"Quality: ",quality)
#     Print_Quality = quality
    return sqrt_error,quality

def assignCluster():
    flag = False
    for i in range(len(dataset)):
        min_val = math.inf
        min_idx = -1
        for j in range(len(centroids)):
            distance = dist(dataset.iloc[i], pd.DataFrame(centroids[j],index=[0]))
            if distance < min_val:
                min_val = distance
                min_idx = j
        if flag==False and cluster_index[i] != min_idx:
            print
            flag = True
        cluster_index[i] = min_idx
    return flag

def newCentroids():
    for i in range(len(centroids)):
        List = [j for j in range(len(dataset)) if cluster_index[j]==i]
#         print("Len:",len(List))
        clustered_data = dataset.iloc[List]
        if (len(clustered_data)>0):
            for column in dataset.columns:
                centroids[i][column] = clustered_data[column].mean()


def bcubed():
    if len(classes) == 0:
        print("no labels")
        return
    cluster_label_combo = np.zeros([len(centroids), len(classes)])
    cluster_count = np.zeros([len(centroids)])
    for i in range(len(dataset)):
        cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]] += 1.0
        cluster_count[cluster_index[i]] += 1.0

    bcp = 0.0
    bcr = 0.0
    for i in range(len(dataset)):
        bcp += cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]]/cluster_count[cluster_index[i]]
        bcr += cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]]/label_count[classes[tuple_labels[i]]]
    bcp /= len(dataset)
    bcr /= len(dataset)
    print("BCubed Precision :", bcp, ", Recall :", bcr)
    return bcp,bcr

def showGraph(loop_num):
    x = []
    y = []
    color = ["red","green", "blue", "yellow", "black"]
    for i in range(len(centroids)+1):
        x.append([])
        y.append([])
    for i in range(len(dataset)):
        x[cluster_index[i]].append(dataset.iloc[i]['x'])
        y[cluster_index[i]].append(dataset.iloc[i]['y'])
    for i in range(len(centroids)):
        x[len(centroids)].append(centroids[i]['x'])
        y[len(centroids)].append(centroids[i]['y'])
    for i in range(len(centroids)):
        plt.scatter(x[i],y[i],color=color[i])
    print("Centroids: ")
    print(x[-1],y[-1])
    
    plt.scatter(x[-1],y[-1],color="black")
    filename = "fig_"+str(loop_num)+".png"
#     print(filename)
    plt.savefig(filename)
    plt.show()




dataset_name = 'ILPD'
k = 2
groundTruth = True
if groundTruth:
    dataset,classes,tuple_labels,label_count = getData(dataset_name)
    bcp = []
    bcr = []
else:
    filepath="Data/"+dataset_name+".data"
    dataset = pd.read_csv(filepath)
dataset = dataset.dropna()
min_max_scaler = MinMaxScaler()
value_attributes = list(dataset.columns)
dataset[value_attributes] = min_max_scaler.fit_transform(dataset[value_attributes])
silhouette_avg=[]
variance = []

for _ in range(5):
    print("_______________________  Instance:",_,"_____________________")
    centroids = []
    cluster_index = [-1]*len(dataset)
    # print(cluster_index)
    for i in range (k): 
        centroid = {}
        random_number = random.randint(0,len(dataset)-1)
        for column in dataset.columns:
            centroid[column] = dataset.iloc[random_number][column]
        centroids.append(centroid)
    loop_num = 0

    while True:
        loop_num += 1
        
        continue_loop = assignCluster()
    #     print("Silhouette average: ", silhouette_avg)
        newError,Print_Quality = calculateError()
        print("Iteration",loop_num,"Error: ",newError)
        newCentroids()
        if continue_loop == False or loop_num>20:
            break

    silhouette_avg.append(silhouette_score(dataset.values.tolist(),cluster_index))
    variance.append(newError)
    print("Average of Variance and Silhouette Coefficient")
    print(str(str(mean(variance)))+","+str(mean(silhouette_avg)))
    if groundTruth:
        precision,recall = bcubed()
        bcp.append(precision); bcr.append(recall)
        print("BCubed precision:",mean(bcp),"BCubed Recall:",mean(bcr))

print("Average of Variance and Silhouette Coefficient over 5 iteration")
print(str(str(mean(variance)))+","+str(mean(silhouette_avg)))
if groundTruth:
    print("BCubed precision:",mean(bcp),"BCubed Recall:",mean(bcr))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
/home/ahasan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


_______________________  Instance: 0 _____________________
Iteration 1 Error:  242.44607232911028
Iteration 2 Error:  192.01708510030656
Iteration 3 Error:  185.02162686886473
Iteration 4 Error:  180.82939477960542
Iteration 5 Error:  178.56793566940675
Iteration 6 Error:  177.07603756109847
Iteration 7 Error:  176.25990792295096
Iteration 8 Error:  176.0075160130971
Iteration 9 Error:  175.95105111983136
Iteration 10 Error:  175.94874121431866
Average of Variance and Silhouette Coefficient
175.94874121431866,0.2491465574817626
BCubed Precision : 0.5911898015663225 , Recall : 0.49788771898212697
BCubed precision: 0.5911898015663225 BCubed Recall: 0.49788771898212697
_______________________  Instance: 1 _____________________
Iteration 1 Error:  253.09947397054472
Iteration 2 Error:  178.43556098686878
Iteration 3 Error:  176.83882054611777
Iteration 4 Error:  176.19090899077412
Iteration 5 Error:  176.12751971562827
Iteration 6 Error:  176.04467681387374
Iteration 7 Error:  175.97334443

In [2]:
print(dataset.iloc[[1,2,4,10]])

           Area  Perimeter  Compactness  L_kernel  W_kernel  Coefficient  \
14.88  0.446281   0.662432     0.368806  0.501069  0.033497     0.215165   
14.29  0.347107   0.879310     0.220721  0.503920  0.256149     0.150665   
16.14  0.533058   0.864791     0.427365  0.664291  0.078133     0.322994   
14.03  0.361570   0.648820     0.303491  0.406985  0.126081     0.237322   

       L_groove  
14.88       0.0  
14.29       0.0  
16.14       0.0  
14.03       0.0  


In [3]:
print(math.inf>=100000000)

True


In [4]:
dataset.iloc[7]

Area           0.630165
Perimeter      0.604356
Compactness    0.649775
L_kernel       0.595153
W_kernel       0.168863
Coefficient    0.668636
L_groove       0.000000
Name: 16.63, dtype: float64

In [29]:
for i in range(20):
    print(random.randint(0,21))

2
9
21
10
9
5
2
20
4
9
15
5
1
2
19
17
12
1
7
19


In [55]:
%reset

import pandas as pd
import numpy as np
import random
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  silhouette_score
from statistics import mean 
from matplotlib import pyplot as plt
# random.seed(5)



def getData(dataset_name):
    attribute_file_name = 'Data/'+dataset_name+".attribute"
    dataset_file_name = 'Data/'+dataset_name+".data"
    att = pd.read_csv(attribute_file_name,
                      delim_whitespace=True,
                     header = None)
    attributes = {rows[0]:rows[1] for _,rows in att.iterrows()}
    dataset = pd.read_csv(dataset_file_name,
                      names=list(attributes.keys()))
    if 'class' in attributes: 
        tuple_labels = []
        label_count = []
        classes = {}
        tuple_labels = list(dataset["class"])
        classes_unique, label_count = np.unique(tuple_labels, return_counts=True)
        idx = 0
        for class_ in classes_unique:
            classes[class_] = idx
            idx += 1
#         print(classes)
        del attributes['class']; del dataset['class']
    return  dataset,classes,tuple_labels,label_count



def dist(x1,x2, minkowski = 2):
    val = 0.0
#     print("In error: ",attributes)
    for att in dataset.columns:
#         if attributes[att]=='value':
        val += (x1[att] - x2[att])*(x1[att] - x2[att])
    val = math.sqrt(val)
    return val

def calculateError():
    error = 0
    sqrt_error = 0
    quality = 0
    for i in range(len(dataset)):
        distance = dist(dataset.iloc[i], pd.DataFrame(centroids[cluster_index[i]],index=[0]))
        sqrt_error += distance
        error += distance
        quality += (1-distance)
#     print("Error:",error,"Squared Error:", sqrt_error,"Quality: ",quality)
#     Print_Quality = quality
    return sqrt_error,quality

def assignCluster():
    flag = False
    for i in range(len(dataset)):
        min_val = math.inf
        min_idx = -1
        for j in range(len(centroids)):
            distance = dist(dataset.iloc[i], pd.DataFrame(centroids[j],index=[0]))
            if distance < min_val:
                min_val = distance
                min_idx = j
        if flag==False and cluster_index[i] != min_idx:
            print
            flag = True
        cluster_index[i] = min_idx
    return flag

def newCentroids():
    for i in range(len(centroids)):
        List = [j for j in range(len(dataset)) if cluster_index[j]==i]
#         print("Len:",len(List))
        clustered_data = dataset.iloc[List]
        if (len(clustered_data)>0):
            for column in dataset.columns:
                centroids[i][column] = clustered_data[column].mean()


def bcubed():
    if len(classes) == 0:
        print("no labels")
        return
    cluster_label_combo = np.zeros([len(centroids), len(classes)])
    cluster_count = np.zeros([len(centroids)])
    for i in range(len(dataset)):
        cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]] += 1.0
        cluster_count[cluster_index[i]] += 1.0

    bcp = 0.0
    bcr = 0.0
    for i in range(len(dataset)):
        bcp += cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]]/cluster_count[cluster_index[i]]
        bcr += cluster_label_combo[cluster_index[i]][classes[tuple_labels[i]]]/label_count[classes[tuple_labels[i]]]
    bcp /= len(dataset)
    bcr /= len(dataset)
    print("BCubed Precision :", bcp, ", Recall :", bcr)
    return bcp,bcr

def showGraph(loop_num):
    x = []
    y = []
    color = ["red","green", "blue", "yellow", "black"]
    for i in range(len(centroids)+1):
        x.append([])
        y.append([])
    for i in range(len(dataset)):
        x[cluster_index[i]].append(dataset.iloc[i]['x'])
        y[cluster_index[i]].append(dataset.iloc[i]['y'])
    for i in range(len(centroids)):
        x[len(centroids)].append(centroids[i]['x'])
        y[len(centroids)].append(centroids[i]['y'])
    for i in range(len(centroids)):
        plt.scatter(x[i],y[i],color=color[i])
    print("Centroids: ")
    print(x[-1],y[-1])
    
    plt.scatter(x[-1],y[-1],color="black")
    filename = "fig_"+str(loop_num)+".png"
#     print(filename)
    plt.savefig(filename)
    plt.show()




dataset_name = 'ILPD'
k = 3
groundTruth = True
if groundTruth:
    dataset,classes,tuple_labels,label_count = getData(dataset_name)
    bcp = []
    bcr = []
else:
    filepath="Data/"+dataset_name+".data"
    dataset = pd.read_csv(filepath)
dataset = dataset.dropna()
min_max_scaler = MinMaxScaler()
value_attributes = list(dataset.columns)
dataset[value_attributes] = min_max_scaler.fit_transform(dataset[value_attributes])
silhouette_avg=[]
variance = []

for _ in range(2,11):
    k = _
#     print("_______________________  Instance:",_,"_____________________")
    centroids = []
    cluster_index = [-1]*len(dataset)
    # print(cluster_index)
    for i in range (k): 
        centroid = {}
        random_number = random.randint(0,len(dataset)-1)
        for column in dataset.columns:
            centroid[column] = dataset.iloc[random_number][column]
        centroids.append(centroid)
    loop_num = 0

    while True:
        loop_num += 1
        
        continue_loop = assignCluster()
    #     print("Silhouette average: ", silhouette_avg)
        newError,Print_Quality = calculateError()
#         print("Iteration",loop_num,"Error: ",newError)
        newCentroids()
        if continue_loop == False or loop_num>20:
            break

    print(str(k)+","+str(newError))
#     silhouette_avg.append(silhouette_score(dataset.values.tolist(),cluster_index))
#     variance.append(newError)
#     print("Average of Variance and Silhouette Coefficient")
#     print(str(str(mean(variance)))+","+str(mean(silhouette_avg)))
#     if groundTruth:
#         precision,recall = bcubed()
#         bcp.append(precision); bcr.append(recall)
#         print("BCubed precision:",mean(bcp),"BCubed Recall:",mean(bcr))

# print("Average of Variance and Silhouette Coefficient over 5 iteration")
# print(str(str(mean(variance)))+","+str(mean(silhouette_avg)))
# if groundTruth:
#     print("BCubed precision:",mean(bcp),"BCubed Recall:",mean(bcr))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
/home/ahasan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


2,175.99434121225957
3,161.91761890995173
4,157.7101566411889
5,146.10299564113262
6,133.64309040453176
7,129.8011272608027
8,127.1044169440692
9,122.689393439627
10,121.45494427640085
